# ITMAL Exercise

REVISIONS|
---------|------------------------------------------------
2018-1219| CEF, initial.                  
2018-0206| CEF, updated and spell checked. 
2018-0208| CEF, minor text updata. 

## Implementing a dummy classifier with fit-predict interface

We begin with the MNIST data-set and will reuse the data loader you just created. Next we create a dummy classifier, and compare the results of the SGD and dummy classifiers using the MNIST data...

#### Qa  Add a Stochastic Gradient Decent [SGD] Classifier

Create a train-test data-set for MNIST and then add the `SGDClassifier` as done in [HOLM], p82.

Split your data and run the fit-predict for the classifier using the MNIST data.

Notice that you have to reshape the MNIST X-data to be able to use the classifier. It may be a 3D array, consisting of 70000 (28 x 28) images, or just a 2D array consisting of 70000 elements of size 784.

A simple `reshape()` could fix this on-the-fly:
```python
X, y = MNIST_GetDataSet()

print("X.shape=",X.shape) # print X.shape= (70000, 28, 28)
if X.ndim==3:
    print("reshaping X..")
    assert y.ndim==1
    X = X.reshape((X.shape[0],X.shape[1]*X.shape[2]))
assert X.ndim==2
print("X.shape=",X.shape) # X.shape= (70000, 784)
```

Remember to use the category-5 y inputs

```python
y_train_5 = (y_train == 5)    
y_test_5  = (y_test == 5)
```
instead of the `y`'s you are getting out of the dataloader...

Test your model on using the test data, and try to plot numbers that have been categorized correctly. Then also find and plots some misclassified numbers.

In [3]:
# TODO: Qa...

#not best practise...
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


import sys, os
import numpy as np
sys.path.append(os.path.join(os.path.curdir, os.path.pardir))
from libitmal.dataloaders import MNIST_GetDataSet
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import SGDClassifier

X, y = MNIST_GetDataSet()


print("X.shape=",X.shape) # print X.shape= (70000, 28, 28)
if X.ndim==3:
    print("reshaping X..")
    assert y.ndim==1
    X = X.reshape((X.shape[0],X.shape[1]*X.shape[2]))
assert X.ndim==2
print("X.shape=",X.shape) # X.shape= (70000, 784)

X_train, X_test, y_train, y_test = train_test_split(X,y)

y_train_5 = (y_train == 5)    
y_test_5  = (y_test == 5)

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train,y_train_5)

y_test_predict = sgd_clf.predict(X_test)

from sklearn.metrics import confusion_matrix
cf = confusion_matrix(y_test_5,y_test_predict)
print(cf)

from libitmal.dataloaders import MNIST_PlotDigit

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12


#i = 0
#while i < 5:
   # print("ok")
    #if y_test_5[i] == y_test_predict[i]:
        #MNIST_PlotDigit(X_test[i])
        #i+=1
#plt.show()
#print("ok")
   



X.shape= (70000, 28, 28)
reshaping X..
X.shape= (70000, 784)
[[15592   364]
 [  255  1289]]


#### Qb Implement a dummy binary classifier

Follow the code found in [HOML], p84, but name you estimator `DummyClassifier` instead of `Never5Classifyer`.

Here our Python class knowledge comes into play. The estimator class hierarchy looks like

<img src="Figs/class_base_estimator.png" style="width:500px">

All Scikit-learn classifiers inherit form `BaseEstimator` (and possible also `ClassifierMixin`), and they must have a `fit-predict` function pair (strangely not in the base class!) and you can actually find the `sklearn.base.BaseEstimator` and `sklearn.base.ClassifierMixin` python source code somewhere in you anaconda install dir, if you should have the nerves to go to such interesting details.

But surprisingly you may just want to implement a class that contains the `fit-predict` functions, ___without inheriting___ from the `BaseEstimator`, things still work due to the pythonic 'duck-typing': you just need to have the class implement the needed interfaces, obviously `fit()` and `predict()` but also the more obscure `get_params()` etc....then the class 'looks like' a `BaseEstimator`...and if it looks like an estimator, it _is_ an estimator (aka. duct typing).

Templates in C++ also allow the language to use compile-time duck typing!

> https://en.wikipedia.org/wiki/Duck_typing

Call the fit-predict on a newly instantiated `DummyClassifier` object, and try to compare the confusion matrix for both the dummy and SDG classifier.

We will be discussing the confusion matrix next, but first, print the `y_test_5.shape` and count the numbers of `y_test_5==True` and `y_test_5==False` and see if you can find these numbers in the confusion matrix.

In [5]:
from sklearn.model_selection import cross_val_score
from libitmal import utils

# TODO: Qb
from sklearn.base import BaseEstimator

class DummyClassifier(BaseEstimator):
    def fit(self,X,y=None):
        pass
    def predict(self,X):
        return np.zeros((len(X),1),dtype=bool)

dummy_classifier = DummyClassifier()
dummy_classifier.fit(X_train)

y_dummy_predicted = dummy_classifier.predict(X_test)




# Inspiration for running cross evaluation on a model, and printing the confusion matrix:
# c=cross_val_score(model, X_test, y_test_5, cv=3, scoring="accuracy")
# print("c=",c)
#M=confusion_matrix(y_test_true, y_test_pred)
# itmalutils.PrintMatrix(M,"M=")

print("DummyClassifier")
cd=cross_val_score(dummy_classifier, X_test, y_test_5, cv=3, scoring="accuracy")
print("c=",cd)
M=confusion_matrix(y_test_5, y_dummy_predicted)
utils.PrintMatrix(M,"M=")

print("SGD classifier")
csgd=cross_val_score(sgd_clf, X_test, y_test_5, cv=3, scoring="accuracy")
print("c=",csgd)
M=confusion_matrix(y_test_5, y_test_predict)
utils.PrintMatrix(M,"M=")

#This shows why accuracy wouldnt be the preffered performance measure for classifiers,
#as the you both would be able to figure out from the dummy classifiers code, but also see from the confusion matricies,
#which clearly show that there arent any true-positives in the dummy-classfier, among other relevant information.

ytestshape = y_test_5.shape[0]
print(f"y_test_5.shape = {ytestshape}")
true = np.count_nonzero(y_test_5 == True)
print(f"Number of y_test_5==True:{true}")
false = ytestshape - true
print(f"Number of y_test_5==False:{false}")
#which obviously matches perfectly with the results of the dummyclassifier...

DummyClassifier
c= [0.91566678 0.90896623 0.91068061]
M=[[15956     0]
   [ 1544     0]]
SGD classifier
c= [0.96606102 0.9026397  0.96141975]
M=[[15592   364]
   [  255  1289]]
y_test_5.shape = 17500
Number of y_test_5==True:1544
Number of y_test_5==False:15956
